# Generating Embeddings with Gemini

This notebook generates embeddings for chatbot conversations.  
Steps:
1. Load JSON files containing cleaned conversations.  
2. Extract message text.  
3. Use OpenAI's embedding model to create vector representations.  
4. Save the embeddings to new JSON files for later use.  

### 1. Import required libraries
We use:
- `genai` for embeddings.  
- `os` and `json` for file handling.  

In [2]:
import google.generativeai as genai
import os
import json

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Configure API key and paths
We define:
- The folder with the input JSON files (`processed_json`).  
- The folder where embeddings will be saved (`embeddingopenAIFiles`). 

In [3]:
genai.configure(api_key="YOUR-API-KEY")

In [9]:
folder_path = "../raw data/processed_json"
out_path = "embeddingGeminiFiles"
os.makedirs(out_path, exist_ok=True)

### 3. Define embedding model
We use `embedding-001` with 768 dimensions. 

In [5]:
EMBEDDING_MODEL = "models/embedding-001"

### 4. Function to generate an embedding
This function calls OpenAI’s embedding API with the selected model and dimensions.

In [ ]:
def generar_embedding(texto):
    try:
        response = genai.embed_content(
            model=EMBEDDING_MODEL,
            content=texto,
            task_type="semantic_similarity",  # puedes cambiarlo según necesidad
        )
        return response['embedding']
    except Exception as e:
        print(f"❌ Error con Gemini embedding: {e}")
        return None

### 5. List input files
We check which JSON files exist in the input folder.
For testing, we only take the first 10 files.5. List input files

### 6. Process each file
For every JSON file:
1. Open and parse the content.  
2. Extract message texts.  
3. Generate embeddings for each text.  
4. Save the embeddings into a new JSON file. 

In [8]:
print("📁 Carpeta de entrada:", os.path.abspath(folder_path))

try:
    files = sorted(os.listdir(folder_path))[:10]
except FileNotFoundError:
    print(f"❌ La carpeta {folder_path} no existe.")
    exit()

print("📄 Archivos detectados:", files)

# ✅ Procesar archivos
for name_file in files:
    ruta = os.path.join(folder_path, name_file)

    if not name_file.endswith('.json'):
        continue

    if not os.path.isfile(ruta):
        print(f"⚠️ Archivo no encontrado (saltado): {ruta}")
        continue

    with open(ruta, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
        except Exception as e:
            print(f"❌ Error leyendo {name_file}: {e}")
            continue

    textos = []

    # ✅ Extraer los textos de los mensajes
    if isinstance(data, dict) and 'messages' in data:
        textos = [msg['content'] for msg in data['messages']
                  if isinstance(msg, dict) and 'content' in msg and isinstance(msg['content'], str)]

    if not textos:
        print(f"⚠️ No se encontraron textos en {name_file}")
        continue

    embeddings = []
    for t in textos:
        emb = generar_embedding(t)
        if emb:
            embeddings.append(emb)

    if not embeddings:
        print(f"⚠️ No se generaron embeddings en {name_file}")
        continue

    nombre_salida = os.path.splitext(name_file)[0] + "_embeddings.json"
    ruta_salida = os.path.join(out_path, nombre_salida)

    with open(ruta_salida, 'w', encoding='utf-8') as f:
        json.dump(embeddings, f)

    print(f"✅ Procesado y guardado: {ruta_salida} ({len(embeddings)} vectores)")

📁 Carpeta de entrada: /mnt/batch/tasks/shared/LS_root/mounts/clusters/elena1/code/Users/elena/elena/raw data/processed_json
📄 Archivos detectados: ['.amlignore', '.amlignore.amltmp', '10ER3iMDW5w4lIlSnHBrIq-in.json', '10er3imdw5w4lilsnhbriq-in.json.amltmp', '111jaycyxHF58KEVIsa0kA-in.json', '11fDRFTlZAPCMeBqTSeYmg-in.json', '12EdU3GcQf2AgjlcgTLb4R-in.json', '12UHWJ1DhbmBIcfVJlbSPK-in.json', '13KMJrTS9SO2aIA8r7SNCj-in.json', '147nMJUjrHyIxKu9Aoka4U-in.json']


FileNotFoundError: [Errno 2] No such file or directory: 'embeddingGeminiFiles/10ER3iMDW5w4lIlSnHBrIq-in_embeddings.json'